In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

In [ ]:
# Read data and remove nan and irrelevant columns
df = pd.read_csv("./Data/2018/LCS/NA LCS 2018 Spring Playoffs - Team Stats - OraclesElixir.csv")
data = df.drop(['STL', 'Player', 'Team', 'Pos', 'D%P15'], axis = 1)
df